In [64]:
import os
import pandas as pd
from tqdm import tqdm
from classes.vqa_model_predictor import DefaultVqaModelPredictor
from evaluate.VqaMedEvaluatorBase import VqaMedEvaluatorBase
from common.utils import VerboseTimer
import vqa_logger 
import logging
from pathlib import Path
import datetime
logger = logging.getLogger(__name__)

In [8]:
%%capture
mp = DefaultVqaModelPredictor.get_contender()

datasets = {'test':mp.df_test, 'validation':mp.df_validation}
df_name_to_predict = 'test'


predictions = {}

for name, df in datasets.items():
    with VerboseTimer(f"Predictions for VQA contender {name}"):
        df_predictions = mp.predict(df)
        predictions[name] = df_predictions



predictions['validation'][:5]

[2019-03-31 20:03:04][common.utils][DEBUG] Starting 'Loading  VQA contender'
[2019-03-31 20:03:04][classes.vqa_model_predictor][DEBUG] Getting model #5 (post_concat_dense_units: 8;
Optimizer: RMSprop
loss: cosine_proximity
activation: sigmoid
prediction vector: answers
lstm_units: 0
batch_size: 64

model_location = 'C:\Users\Public\Documents\Data\2019\models\20190223_2239_45\vqa_model.h5')
[2019-03-31 20:03:04][common.utils][DEBUG] Starting 'Loading Model'
[2019-03-31 20:03:07][common.utils][DEBUG] Loading Model: 0:00:03.083447
[2019-03-31 20:03:07][classes.vqa_model_predictor][DEBUG] Getting model #72 (prediction_vector: words
 question_category: Abnormality
augmentations: 20 
epochs: 10
batch size: 64
based on model 5)
[2019-03-31 20:03:07][common.utils][DEBUG] Starting 'Loading Model'
[2019-03-31 20:03:10][common.utils][DEBUG] Loading Model: 0:00:02.854339
[2019-03-31 20:03:10][root][DEBUG] For Abnormality, got specialized model (DB: 72, Folder: ModelFolder(folder="C:\Users\Public\D

In [9]:
outputs = {}
for name, df_predictions in predictions.items():
    curr_predictions = df_predictions.prediction.values
    df_predicted = datasets[name]
    df_output = df_predicted.copy()
    df_output['image_id'] = df_output.path.apply(lambda p: p.rsplit(os.sep)[-1].rsplit('.', 1)[0])
    df_output['prediction'] = curr_predictions

    columns_to_remove = ['path',  'answer_embedding', 'question_embedding', 'group', 'diagnosis', 'processed_answer']
    for col in columns_to_remove:
        del df_output[col]

    sort_columns = sorted(df_output.columns, key=lambda c: c not in ['question', 'prediction', 'answer'])
    df_output = df_output[sort_columns]    
    outputs[name] = df_output

In [12]:
df_output_test = outputs['test']
df_output_validation = outputs['validation']

In [13]:
df_output_validation.sample(10)

,question,answer,prediction,image_name,processed_question,question_category,image_id
14579,what is the primary abnormality in this image?,primary sclerosing cholangitis,multiple carcinoma osteomyelitis angioma cyst,synpic21890.jpg,what is the primary abnormality in this image,Abnormality,synpic21890
13887,what part of the body does this x-ray show?,"lung, mediastinum, pleura",skull and contents,synpic46876.jpg,what part of the body does this ray show,Organ,synpic46876
14571,what is the primary abnormality in this image?,epidermoid cyst,sclerosis tuberous toxoplasmosis pituitary apo...,synpic33107.jpg,what is the primary abnormality in this image,Abnormality,synpic33107
14117,what organ system is displayed in this ct scan?,skull and contents,skull and contents,synpic53024.jpg,what organ system is displayed in this ct scan,Organ,synpic53024
14333,what is most alarming about this ct scan?,sialolithiasis,fracture dysplasia disease bone fractures,synpic52140.jpg,what is most alarming about this ct scan,Abnormality,synpic52140
13453,in what plane is this x-ray?,pa,pa,synpic39557.jpg,in what plane is this ray,Plane,synpic39557
14602,what abnormality is seen in the image?,"sequestration, intralobar bronchopulmonary",pulmonary syndrome aortic embolism hepatic,synpic59337.jpg,what abnormality is seen in the image,Abnormality,synpic59337
13130,is this a t1 weighted image?,no,yes,synpic58993.jpg,is this t1 weighted image,Modality,synpic58993
12893,is this a contrast or noncontrast ct?,noncontrast,contrast,synpic20729.jpg,is this contrast or noncontrast ct,Modality,synpic20729
14791,what is most alarming about this mri?,"spine, tethered cord",spinal spine cyst epidural cervical,synpic48459.jpg,what is most alarming about this mri,Abnormality,synpic48459


In [11]:
mp

VqaModelPredictor(model=5, specialized_classifiers={'Abnormality': 72, 'Modality': 69, 'Organ': 70, 'Plane': 71})

In [16]:
len(df_output_test), len(df_output_test.image_id.drop_duplicates())

(500, 500)

In [25]:
def get_str(df):
    strs = []
    debug_output_rows = df.apply(lambda row: row.image_id + '|'+ row.question + '|'+ row.prediction, axis=1 )
    output_rows = df.apply(lambda row: row.image_id + '|'+ row.prediction + '|'+row.answer, axis=1 )
    output_rows = output_rows.str.strip('|')
    rows = output_rows.values
    res = '\n'.join(rows)
    return res

res = get_str(df_output_test)
#res_val = get_str(df_output_validation)


In [26]:
print(res[:200])

synpic54082|cta ct angiography
synpic48556|no
synpic50696|iv
synpic37194|us doppler ultrasound
synpic31308|an angiogram
synpic24739|t1
synpic32365|yes
synpic47936|iv
synpic29792|yes
synpic43561|no
syn


Get evaluations per category:

In [70]:
evaluations = {}
pbar = tqdm(df_output_validation.groupby('question_category'))
for question_category, df in pbar:        
    pbar.set_description(f'evaluating {len(df)} for {question_category} items')
    predictions = df.prediction.values
    ground_truth = df.answer.values
    curr_evaluations = VqaMedEvaluatorBase.get_all_evaluation(predictions=predictions, ground_truth=ground_truth)
    evaluations[question_category] = curr_evaluations    


evaluating 500 for Plane items: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:09<00:00,  3.03s/it]


In [85]:
evaluations
df_evaluations = pd.DataFrame(evaluations).T.sort_values(by='bleu')
df_evaluations

,bleu,wbss
Abnormality,0.166595,0.152170
Organ,0.708409,0.699596
Plane,0.739682,0.743336
Modality,0.763459,0.713040


In [102]:
model_repr = repr(mp)
model_repr
sub_models = {category: folder for category, (model, folder) in mp.model_by_question_category.items()}
sub_models_str = '\n'.join([str(f'{category}: {folder} ({folder.prediction_data_name})'.replace('\\','\\\\')) for category, folder in sub_models.items()])

model_description_copy = df_evaluations.copy()
model_description_copy['prediction_vector'] = model_description_copy.index.map(lambda category:sub_models[category].prediction_data_name)


model_description =f'''
==Model==
{model_repr}

==Submodels==
{sub_models_str}

==validation evaluation==
{model_description_copy.to_string()}
'''


print(model_description)




==Model==
VqaModelPredictor(model=5, specialized_classifiers={'Abnormality': 72, 'Modality': 69, 'Organ': 70, 'Plane': 71})

==Submodels==
Abnormality: ModelFolder(folder="C:\\Users\\Public\\Documents\\Data\\2019\\models\\20190329_0854_26") (words)
Modality: ModelFolder(folder="C:\\Users\\Public\\Documents\\Data\\2019\\models\\20190329_0133_06") (answers)
Organ: ModelFolder(folder="C:\\Users\\Public\\Documents\\Data\\2019\\models\\20190329_0440_18") (answers)
Plane: ModelFolder(folder="C:\\Users\\Public\\Documents\\Data\\2019\\models\\20190329_0647_26") (answers)

==validation evaluation==
                 bleu      wbss prediction_vector
Abnormality  0.166595  0.152170             words
Organ        0.708409  0.699596           answers
Plane        0.739682  0.743336           answers
Modality     0.763459  0.713040           answers



In [103]:
import time
now = time.time()
ts = datetime.datetime.fromtimestamp(now).strftime('%Y%m%d_%H%M_%S')
submission_base_folder = Path('C:\\Users\\Public\\Documents\\Data\\2019\\submissions')
submission_folder = submission_base_folder/ts
submission_folder.mkdir()


txt_path = submission_folder/f'submission_{ts}.txt'
txt_path.write_text(res)


model_description_path = submission_folder/f'model_description.txt'
model_description_path.write_text(model_description)




848